### Crude and age-standardized death rates for Uganda and USA 

To calculate the crude population death rate I obtained 2019 age-grouped population data for the USA and Uganda from UN World Population Prospects. The age groups included some groups not present in the provided death rate data, so I aggregated data for groups over 85 y.o. into one group for consistency. I merged the population estimates and death rate datasets on age group and calculated crude deaths by multiplying death rate by population in each age group. Then, I divided the total number of deaths across all groups by the total population number resulting in a crude COPD death rate for the whole population.

For calculating the standardised death rate, which makes comparing different countries or different periods of time more meaningful, I extracted the new WHO standard data from the recommended paper. I used this data together with the table with age-specific death rates for the country to calculate the weighted sum. For each age group, I multiplied the age-specific death rate by the proportion of the WHO standard population in that age group and summed these values across all age groups. I then divided the weighted sum by the total sum of the WHO standard population weights.

The effect of using the standardised data is quite striking. When looking at raw data, age-specific death rates are higher in Uganda than in the USA by an order of magnitude for children and younger people. Death rates in the two countries become more similar with increasing age. Surprisingly, the crude measure showed a higher total death rate for the USA and not for Uganda. This effect disappeared when comparing standardised measures. Uganda is demographically interesting because of its unusually high proportion of children (a result of high fertility rate). Using the WHO standard summary measure in this case allowed for the comparison of mortality rates between populations with different age structures. Without the standardisation, the high proportion of children in Uganda was diluting the COPD mortality rate, making the total death rate difficult to compare to countries with other age structures.


### 1. Crude deaths calculation

In [1]:
import pandas as pd
# load datasets
deaths_file_location = r'C:\Users\karol\Downloads\age-specific_COPD_deaths.csv'
deaths = pd.read_csv(deaths_file_location)
deaths = deaths.rename(columns = {'Age group (years)':'age_group', 'Death rate, United States, 2019': 'usa_death_rate', 'Death rate, Uganda, 2019':'uganda_death_rate'})

In [2]:
deaths

,age_group,usa_death_rate,uganda_death_rate
0,0-4,0.04,0.40
1,5-9,0.02,0.17
2,10-14,0.02,0.07
3,15-19,0.02,0.23
4,20-24,0.06,0.38
5,25-29,0.11,0.40
6,30-34,0.29,0.75
7,35-39,0.56,1.11
8,40-44,1.42,2.04
9,45-49,4.00,5.51


In [3]:

# Age-grouped population data from https://population.un.org/wpp/Download/Standard/Population/
# I chose Population by Five-year Age Groups - Both Sexes (XLSX, 45.65 MB) for consistency with the other datasets.
# The file was too big to load in Google sheets, so I selected only the first sheet with a short python script pasted below.
# In google sheets I filtered columns by year (2019) and Region : Uganda  and USA. I then transposed the data to have population values in 2 columns, 
# one for USA and one for Uganda

# #######################################

# import pandas as pd

# def extract_first_tab(input_excel_file, output_csv_file):
#     try:
#         # Read the Excel file
#         xls = pd.ExcelFile(input_excel_file)

#         # Get the names of all sheets
#         sheet_names = xls.sheet_names

#         # Extract data from the first sheet
#         first_sheet_data = xls.parse(sheet_names[0])

#         # Save the data as CSV
#         first_sheet_data.to_csv(output_csv_file, index=False)

#         print(f"Successfully extracted and saved the first tab as {output_csv_file}")

#     except Exception as e:
#         print(f"Error: {e}")

# # Specify your input Excel file and output CSV file
# input_excel_file = r'C:\Users\karol\Downloads\WPP2022_POP_F02_1_POPULATION_5-YEAR_AGE_GROUPS_BOTH_SEXES.xlsx'
# output_csv_file = r'C:\Users\karol\Downloads\population_first_tab.csv'

# # Call the function to extract the first tab and save as CSV
# extract_first_tab(input_excel_file, output_csv_file)

# ######################################



# load population estimates
population_file_location = r'C:\Users\karol\Downloads\usa_uganda_population_data_2019.csv'
# Skip rows that contain untransposed data 
rows_to_skip = list(range(5)) + [27]

population = pd.read_csv(population_file_location, skiprows=rows_to_skip, usecols=[0, 3, 4])
population = population.rename(columns = {'Age':'age_group', 'Usa_corrected': 'usa_population', 'Uganda_corrected':'uganda_population'})

In [4]:
population

,age_group,uganda_population,usa_population
0,0-4,7330000.0,19800000.0
1,5-9,6610000.0,20700000.0
2,10-14,5900000.0,22100000.0
3,15-19,5150000.0,21900000.0
4,20-24,4350000.0,21900000.0
5,25-29,3500000.0,23400000.0
6,30-34,2620000.0,22800000.0
7,35-39,1900000.0,22300000.0
8,40-44,1500000.0,20700000.0
9,45-49,1230000.0,21200000.0


In [5]:
# I want to merge the 2 datasets on age group but need to handle the differences in age groups in the 2 datasets.
# The population dataset has age groups up to 100+ while in the deaths dataset, the last group is 85+
# One solution is to aggregate data in population df from the groups not present in deaths df

replacement_dict = {'85-89': '85+', '90-94': '85+', '95-99': '85+', '100+': '85+'}
population['age_group'] = population['age_group'].replace(replacement_dict)

#alternative is to create nans and fill: population['age_group'] = population['age_group'].fillna('85+')

# Sum the values for 'uganda_population' and 'usa_population' for '85+'
total_uganda_population = population.loc[population['age_group'] == '85+', 'uganda_population'].sum()
total_usa_population = population.loc[population['age_group'] == '85+', 'usa_population'].sum()

# Update the values for 'uganda_population' and 'usa_population' for '85+'
population.loc[population['age_group'] == '85+', 'uganda_population'] = total_uganda_population
population.loc[population['age_group'] == '85+', 'usa_population'] = total_usa_population

# Drop duplicate rows for '85+' (keeping only one entry)
population = population.drop_duplicates(subset='age_group', keep='first')

In [6]:
population

,age_group,uganda_population,usa_population
0,0-4,7330000.0,19800000.0
1,5-9,6610000.0,20700000.0
2,10-14,5900000.0,22100000.0
3,15-19,5150000.0,21900000.0
4,20-24,4350000.0,21900000.0
5,25-29,3500000.0,23400000.0
6,30-34,2620000.0,22800000.0
7,35-39,1900000.0,22300000.0
8,40-44,1500000.0,20700000.0
9,45-49,1230000.0,21200000.0


In [7]:
merged_crude = pd.merge(population, deaths, on = 'age_group')
merged_crude

,age_group,uganda_population,usa_population,usa_death_rate,uganda_death_rate
0,0-4,7330000.0,19800000.0,0.04,0.40
1,5-9,6610000.0,20700000.0,0.02,0.17
2,10-14,5900000.0,22100000.0,0.02,0.07
3,15-19,5150000.0,21900000.0,0.02,0.23
4,20-24,4350000.0,21900000.0,0.06,0.38
5,25-29,3500000.0,23400000.0,0.11,0.40
6,30-34,2620000.0,22800000.0,0.29,0.75
7,35-39,1900000.0,22300000.0,0.56,1.11
8,40-44,1500000.0,20700000.0,1.42,2.04
9,45-49,1230000.0,21200000.0,4.00,5.51


In [8]:
# Because the death rate is already given per 100000, I could skip dividing by it and then multiplying again in the next cell.
# It's a bit redundant but I did it here to make sure I know what the death rate reflects in each step.

merged_crude['usa_deaths']= (merged_crude['usa_population']* merged_crude['usa_death_rate'])/100000
merged_crude['uganda_deaths']= (merged_crude['uganda_population']* merged_crude['uganda_death_rate'])/100000

In [9]:
 merged_crude

,age_group,uganda_population,usa_population,usa_death_rate,uganda_death_rate,usa_deaths,uganda_deaths
0,0-4,7330000.0,19800000.0,0.04,0.40,7.9200,29.320000
1,5-9,6610000.0,20700000.0,0.02,0.17,4.1400,11.237000
2,10-14,5900000.0,22100000.0,0.02,0.07,4.4200,4.130000
3,15-19,5150000.0,21900000.0,0.02,0.23,4.3800,11.845000
4,20-24,4350000.0,21900000.0,0.06,0.38,13.1400,16.530000
5,25-29,3500000.0,23400000.0,0.11,0.40,25.7400,14.000000
6,30-34,2620000.0,22800000.0,0.29,0.75,66.1200,19.650000
7,35-39,1900000.0,22300000.0,0.56,1.11,124.8800,21.090000
8,40-44,1500000.0,20700000.0,1.42,2.04,293.9400,30.600000
9,45-49,1230000.0,21200000.0,4.00,5.51,848.0000,67.773000


In [10]:
# Calculate total population and total deaths for each country
total_uganda_population = merged_crude['uganda_population'].sum()
total_usa_population = merged_crude['usa_population'].sum()
total_uganda_deaths = merged_crude['uganda_deaths'].sum()#this is per 100,000 
total_usa_deaths = merged_crude['usa_deaths'].sum()

# check the total is what you would expect for these populations
# total_uganda_population
# total_uganda_deaths

# Calculate death rate per 100,000 for each country
death_rate_uganda = (total_uganda_deaths*100000) / total_uganda_population
death_rate_usa = (total_usa_deaths*100000) / total_usa_population

# Print the results
print(f"Uganda Death Rate per 100,000: {death_rate_uganda:.1f}")
print(f"USA Death Rate per 100,000: {death_rate_usa:.1f}")

Uganda Death Rate per 100,000: 5.8
USA Death Rate per 100,000: 57.3


### 2. Standardised deaths calculation

In [11]:
# load datasets
who_file_location = r'C:\Users\karol\Downloads\WHO_standard_population.csv'
who = pd.read_csv(who_file_location)

who = who.rename(columns = {'Age group':'age_group', 'WHO World Standard*': 'who_standard'})
who

,age_group,who_standard
0,0-4,8.86
1,5-9,8.69
2,10-14,8.60
3,15-19,8.47
4,20-24,8.22
5,25-29,7.93
6,30-34,7.61
7,35-39,7.15
8,40-44,6.59
9,45-49,6.04


In [12]:
# merge with deaths dataset
who_merged = pd.merge(who, deaths, on = 'age_group')

In [13]:
who_merged

,age_group,who_standard,usa_death_rate,uganda_death_rate
0,0-4,8.86,0.04,0.40
1,5-9,8.69,0.02,0.17
2,10-14,8.60,0.02,0.07
3,15-19,8.47,0.02,0.23
4,20-24,8.22,0.06,0.38
5,25-29,7.93,0.11,0.40
6,30-34,7.61,0.29,0.75
7,35-39,7.15,0.56,1.11
8,40-44,6.59,1.42,2.04
9,45-49,6.04,4.00,5.51


In [14]:
# for each age group, multiply the age-specific death rate by the proportion of the WHO standard population 
# in that age group. Sum these values across all age groups. Calculate the ASMR
who_merged['who_death_rate_usa'] = who_merged['usa_death_rate']*who_merged['who_standard']
who_merged['who_death_rate_uganda'] = who_merged['uganda_death_rate']*who_merged['who_standard']
who_merged

,age_group,who_standard,usa_death_rate,uganda_death_rate,who_death_rate_usa,who_death_rate_uganda
0,0-4,8.86,0.04,0.40,0.3544,3.5440
1,5-9,8.69,0.02,0.17,0.1738,1.4773
2,10-14,8.60,0.02,0.07,0.1720,0.6020
3,15-19,8.47,0.02,0.23,0.1694,1.9481
4,20-24,8.22,0.06,0.38,0.4932,3.1236
5,25-29,7.93,0.11,0.40,0.8723,3.1720
6,30-34,7.61,0.29,0.75,2.2069,5.7075
7,35-39,7.15,0.56,1.11,4.0040,7.9365
8,40-44,6.59,1.42,2.04,9.3578,13.4436
9,45-49,6.04,4.00,5.51,24.1600,33.2804


In [15]:
# Divide the weighted sum by the total sum of the WHO standard population weights.

# Calculate total population and total deaths for each country
total_weighted_death_rate_usa = who_merged['who_death_rate_usa'].sum()
total_weighted_death_rate_uganda = who_merged['who_death_rate_uganda'].sum()
total_who_weights = who_merged['who_standard'].sum()

# Calculate death rate per 100,000 for each country
who_death_rate_usa = total_weighted_death_rate_usa / total_who_weights
who_death_rate_uganda = total_weighted_death_rate_uganda / total_who_weights

# Print the results
print(f"Uganda Death Rate WHO corrected per 100,000: {who_death_rate_uganda:.1f}")
print(f"USA Death Rate WHO corrected per 100,000: {who_death_rate_usa:.1f}")

Uganda Death Rate WHO corrected per 100,000: 28.7
USA Death Rate WHO corrected per 100,000: 28.4
